In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outsid
        e of the current session

# Arquitectura recomendada (Kaggle + Colab)
## 1. Kaggle Notebooks — procesamiento pesado

Usar Kaggle para:
* Lectura de datasets grandes (Parquet, CSV > varios GB)
* Limpieza inicial
* Feature engineering
* Agregaciones
* Generación de datasets intermedios

**Ventajas**
* Datasets montados nativamente (sin bugs de disco)
* Más I/O estable
* Entorno reproducible
* Ideal para EDA y pipelines de datos

## 2. Colab — modelamiento y visualización

Usar Colab para:
* Cargar datasets reducidos
* Entrenamiento de modelos
* Gráficos
* Validación y explicación de resultados

**Ventajas**
* GPU/CPU flexibles
* Entorno limpio
* Menos riesgo de errores de disco

## Flujo recomendado (resumen)

Kaggle (raw data)
   
   ↓
   
Kaggle (clean + features)
   
   ↓
   
Parquet reducido
   
   ↓
   
Colab (modelos + plots)
   
   ↓
   
Resultados

## 1. Ingesta de datos (Kaggle)

**Entrada**

* Dataset histórico de vuelos
* Formato: parquet (preferido)
* Ver columnas y tipos

In [1]:
import pandas as pd
import kagglehub

'''
Manualmente en entorno Kaggle
Add Input -> Datasets -> arvindnagaonkar/flight-delay
'''
arvindnagaonkar_flight_delay_path = kagglehub.dataset_download('arvindnagaonkar/flight-delay')

print('Data source import complete.')
print ('Reading data...')
df = pd.read_parquet("/kaggle/input/flight-delay/Flight_Delay.parquet")

df.info()

Data source import complete.
Reading data...
<class 'pandas.core.frame.DataFrame'>
Index: 30132672 entries, 1 to 596675
Data columns (total 29 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Year                       int64  
 1   Month                      int64  
 2   DayofMonth                 int64  
 3   FlightDate                 object 
 4   Marketing_Airline_Network  object 
 5   OriginCityName             object 
 6   DestCityName               object 
 7   CRSDepTime                 int64  
 8   DepTime                    float64
 9   DepDelay                   float64
 10  DepDelayMinutes            float64
 11  TaxiOut                    float64
 12  WheelsOff                  float64
 13  WheelsOn                   float64
 14  TaxiIn                     float64
 15  CRSArrTime                 int64  
 16  ArrTime                    float64
 17  ArrDelay                   float64
 18  ArrDelayMinutes            float64
 19  CR

In [2]:
cols = [
    "OriginCityName",
    "DestCityName",
    "DistanceGroup",
    "Marketing_Airline_Network",
    "Year",
    "Month",
    "DayofMonth",
    "FlightDate",
    "CRSDepTime",
    "DepDelayMinutes",
    "TaxiOut",
    "TaxiIn",
    "CRSElapsedTime",
    "AirTime",
    "Distance",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay",
    "ArrDelay",
    "ArrDelayMinutes"
]

df = df[cols].copy()

## 2. Limpieza inicial

In [4]:
print("Buscando nulos...")
nulls = (
    df
        .isna()
        .sum()
        .to_frame("n_nulls")
        .assign(pct=lambda x: x.n_nulls / len(df))
        .sort_values("pct", ascending=False)
)

nulls

Buscando nulos...


,n_nulls,pct
OriginCityName,0,0.0
DestCityName,0,0.0
DistanceGroup,0,0.0
Marketing_Airline_Network,0,0.0
Year,0,0.0
Month,0,0.0
DayofMonth,0,0.0
FlightDate,0,0.0
CRSDepTime,0,0.0
DepDelayMinutes,0,0.0


### **Duplicados**

*   Eliminar registros repetidos que puedan sesgar los resultados.

In [5]:
print("Buscando duplicados...")
num_duplicates = df.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicates}")

Buscando duplicados...
Número de filas duplicadas: 0


## 3. Feature engineering

* Variable objetivo
* Variables temporales
* Dataset final para modelamiento

### **Formateo**

*  Asegurarse de que las fechas sean tratadas como fechas y no como simple texto.
*  Convertir fecha/hora a datetime.

### **Creación de variables relevantes**

Creación de variables relevantes a partir de los existentes para ayudar al modelo. Extraer `'day_of_year'`, `'week_of_year'` y `'quarter'` de la columna `'FlightDate'` y añadirlos al DataFrame. Estas características pueden capturar la estacionalidad.

In [6]:
# Feature engineering
# variable objectivo
df["delayed"] = (df["ArrDelay"] > 15).astype(int)

# variables temporales
df["FlightDate"] = pd.to_datetime(df["FlightDate"])
df["day_of_week"] = df["FlightDate"].dt.dayofweek
df['day_of_year'] = df['FlightDate'].dt.dayofyear
df['week_of_year'] = df['FlightDate'].dt.isocalendar().week.astype(int)
df['quarter'] = df['FlightDate'].dt.quarter

# Dataset reducido
df_model = df[[
    "ArrDelayMinutes",
    "OriginCityName",
    "DestCityName",
    "DistanceGroup",
    "Marketing_Airline_Network",
    "DayofMonth",
    "Year",
    "Month",
    "day_of_week",
    "CRSDepTime",
    "DepDelayMinutes",
    "TaxiOut",
    "TaxiIn",
    "CRSElapsedTime",
    "AirTime",
    "Distance",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay",
    "day_of_year",
    "week_of_year",
    "quarter",
    "delayed"
]]


Las nuevas características creadas capturan varios patrones relacionados con el tiempo que pueden influir en los retrasos de los vuelos:

*   `day_of_year`: Captura la estacionalidad anual (por ejemplo, temporadas altas de viajes).
*   `week_of_year`: Proporciona una granularidad semanal, útil para detectar tendencias semanales recurrentes.
*   `quarter`: Identifica patrones trimestrales, que pueden correlacionarse con ciclos económicos, días festivos o cambios climáticos estacionales.

Muestreo proporcional por año (estratificado temporal)
1. Definir la proporción a extraer

Ejemplo: 10% de cada año.

In [8]:
SAMPLE_FRAC = 0.10
RANDOM_STATE = 42

2. Muestreo balanceado por año

Esto garantiza:
* Igual proporción por año
* No sesgo temporal
* Reproducibilidad

In [9]:
df_sample = (
    df_model
        .groupby("Year", group_keys=True)
        .apply(
            lambda x: x.sample(
                frac=SAMPLE_FRAC,
                random_state=RANDOM_STATE
            ),
            include_groups=False
        )
        .reset_index(level=0)
)


'''
# Alternativa: número fijo de registros por año 
# (si hay años desbalanceados)

N_PER_YEAR = 50_000

df_sample = (
    df_model
        .groupby("Year", group_keys=False)
        .apply(lambda x: x.sample(
            n=min(len(x), N_PER_YEAR),
            random_state=42
        ))
)
'''

'\n# Alternativa: número fijo de registros por año \n# (si hay años desbalanceados)\n\nN_PER_YEAR = 50_000\n\ndf_sample = (\n    df_model\n        .groupby("Year", group_keys=False)\n        .apply(lambda x: x.sample(\n            n=min(len(x), N_PER_YEAR),\n            random_state=42\n        ))\n)\n'

**Verificación recomendada (rápida)**

In [11]:
df_model["Year"].value_counts(normalize=True)


Year
2019    0.222604
2018    0.213674
2022    0.189364
2021    0.173714
2020    0.137662
2023    0.062983
Name: proportion, dtype: float64

## 4. Persistencia intermedia (Kaggle → Colab)

Guardar solo el dataset reducido:

In [13]:
df_sample.to_parquet(
    "/kaggle/working/df_model.parquet",
    index=False
)

Luego descargar solo **df_model.parquet** (mucho más liviano) y cargarlo en Colab.

### 5. Carga en Colab

En Kaggle *Run -> Kaggle Jupiter Server*. En la parte derecha aparece un botón *Open in Collab*.

Despues, en Collab, subir el archivo al almacenamiento de sessión.

**A method to upload the file directly from a local machine using Python code**

In [ ]:
import os

file_to_delete = 'df_model.parquet'
if os.path.exists(file_to_delete):
    os.remove(file_to_delete)
    print(f'File "{file_to_delete}" deleted successfully.')
else:
    print(f'File "{file_to_delete}" not found.')

In [ ]:
from google.colab import files
import shutil

# This will open a file selection dialog
uploaded = files.upload()

# Iterate through the uploaded files and move df_model.parquet to /content/
for filename in uploaded.keys():
    if filename == 'df_model.parquet':
        shutil.move(filename, '/content/' + filename)
        print(f'File "{filename}" uploaded successfully to /content/.')
    else:
        print(f'Uploaded file "{filename}" is not df_model.parquet. Please upload the correct file.')


In [ ]:
import os

print(os.listdir('/content/'))

### 1. Carga y Exploración Inicial (Profiling)

In [ ]:
import pandas as pd

df = pd.read_parquet("/content/df_model.parquet")

Inspección visual

In [ ]:
df.head()

### **Estructura**

*   Revisar cuántas filas y columnas hay.

In [ ]:
print(f"Cantidad de columnas y filas: {df.shape}")

Qué tipo de datos contiene cada una (¿son números, fechas, texto?)

In [ ]:
df.dtypes

### **Estadística básica**

*   Calcular medias, medianas y desviaciones estándar para entender la distribución de los números.

df.describe()


---

> **Aquí las estadísticas clave que estás viendo**:
>
> - **count (conteo)**: Es el número de observaciones no nulas en cada columna. En este conjunto de datos, todas las columnas numéricas tienen 100,000 entradas, lo que significa que no hay valores faltantes en estas columnas.
>
> - **mean (media)**: Es el valor promedio de cada columna. Por ejemplo, el promedio de `DepDelay` (retraso de salida) es de aproximadamente 10.12 minutos, y la distancia promedio de los vuelos es de alrededor de 764.65 millas.
>
> - **std (desviación estándar)**: Mide la cantidad de variación o dispersión de un conjunto de valores. Una desviación estándar alta indica que los datos están más dispersos, mientras que una baja indica que los datos tienden a estar cerca de la media. Por ejemplo, `DepDelay` tiene una desviación estándar de 49.23, lo que es bastante alto en comparación con su media, sugiriendo que hay una variabilidad significativa en los retrasos de salida.
>
> - **min (mínimo)**: Es el valor mínimo en cada columna. Para `DepDelay`, el mínimo es -45, lo que indica que algunos vuelos salieron 45 minutos antes de lo programado.
>
> - **25% (Q1)**: Es el primer cuartil, lo que significa que el 25% de los datos están por debajo de este valor.
>
> - **50% (Q2)**: Es el segundo cuartil o mediana, que es el valor medio cuando los datos están ordenados. Para `DepDelay`, la mediana es de -3 minutos, lo que implica que al menos la mitad de los vuelos salieron a tiempo o antes.
>
> - **75% (Q3)**: Es el tercer cuartil, lo que significa que el 75% de los datos están por debajo de este valor.
>
> - **max (máximo)**: Es el valor máximo en cada columna. Para `DepDelay`, el máximo es de 1682 minutos, lo que indica que algunos vuelos experimentaron retrasos muy largos.



# **3. Análisis Exploratorio de Datos (EDA)**

Creación de variables relevantes a partir de los existentes para ayudar al modelo.

In [ ]:
import numpy as np

#df["distance_bin"] = pd.qcut(df["Distance"], q=5)
df['CRSDepTime_minutes'] = (df_model['CRSDepTime'] // 100) * 60 + (dfl['CRSDepTime'] % 100)
df['CRSDepTime_sin'] = np.sin(2 * np.pi * df['CRSDepTime_minutes'] / 1440)
df['CRSDepTime_cos'] = np.cos(2 * np.pi * df['CRSDepTime_minutes'] / 1440)
df['distancia_km'] = df['Distance'] * 1.60934
df["hour"] = df["CRSDepTime"] // 100
df['is_weekend'] = ((df['day_of_week'] == 5) | (df['day_of_week'] == 6)).astype(int
days_map = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}

df["day_name"] = df["day_of_week"].map(days_map)



Las nuevas características creadas capturan varios patrones relacionados con el tiempo que pueden influir en los retrasos de los vuelos:

*   `is_weekend`: Distingue entre viajes entre semana y fines de semana, ya que la demanda de viajes y los patrones operativos a menudo difieren significativamente en estos períodos.
*   `CRSDepTime_minutes`, `CRSDepTime_sin` y `CRSDepTime_cos`: La hora de salida programada está mejor representada por las características cíclicas, que manejan la naturaleza cíclica del tiempo de forma más adecuada para los .
*   `distancia_km`: la columna original (en millas) ahora se ha convertido en kilómetros.


## **Detección de Outliers**

Identificar valores atípicos (por ejemplo, un sueldo de un billón de dólares en una lista de empleados comunes) que podrían arruinar el modelo.

In [ ]:
df_numeric = df.select_dtypes(include='number')
print(f"Forma del DataFrame numérico: {df_numeric.shape}")
df_numeric.head()

In [ ]:
print("\nTipos de datos de df_numeric")
print("*********************************\n")
print(df_numeric.dtypes)

## **Ajustar nombres de columnas**

Ajustar nombres de columnas según dataset:

```
{
  "aerolinea": "AZ",
  "origen": "GIG",
  "destino": "GRU",
  "fecha_partida": "2025-11-10T14:30:00",
  "distancia_km": 350
}
```

In [ ]:
#  'FlightDate' -> 'fecha_partida' esta dividida en los dias, numeros del mes, etc.
df_numeric[['origen', 'destino', 'aerolinea']] = df[['OriginCityName', 'DestCityName', 'Marketing_Airline_Network']]

print("Columnas renombradas del DataFrame (primeras 5 filas)")
print("*****************************************************\n")
display(df_numeric[['aerolinea', 'distancia_km', 'origen', 'destino']].head()) # , 'fecha_partida'


## **Contar y mostrar outliers**

*   Contar el número de puntos de datos que están por debajo del límite inferior o por encima del límite superior para cada columna numérica. Mostrar estos conteos para indicar cuántos outliers hay en cada variable.

Para contar el número de outliers en cada columna numérica, itera a través de las columnas en `df_numeric`, aplica los límites inferior y superior definidos para identificar los outliers y almacenaré los conteos en un diccionario.

In [ ]:
outlier_counts = {}

# Calculate Q1, Q3, IQR, lower_bound, and upper_bound within this cell
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

for column in df_numeric.columns:
    # Skip Year and Month columns as they are constant in this dataset sample
    if column in ['Year', 'Month']:
        continue

    # Count outliers for each column
    num_outliers = df_numeric[(df_numeric[column] < lower_bound[column]) | (df_numeric[column] > upper_bound[column])].shape[0]
    outlier_counts[column] = num_outliers

print("Número de outliers por columna numérica")
print("****************************************\n")
for col, count in outlier_counts.items():
    print(f"{col}: {count}")

*   Ahora que se han calculado y mostrado los conteos de outliers, el siguiente paso es visualizar estos outliers para columnas numéricas clave utilizando diagramas de caja. Esto proporcionará una representación gráfica de la distribución y la magnitud de los outliers.

## **Correlaciones**

¿Si la variable A sube, la variable B también? Esto nos dirá qué datos son realmente importantes para la predicción.

A continuación, para cada columna numérica, calculará algo llamado Rango Intercuartílico (IQR). El IQR ayuda a definir límites que nos indican qué tan lejos están los datos de lo que se considera normal. Para calcularlo, se debe encontrar dos valores especiales:

*   Q1: el primer cuartil, que es el valor que separa el 25% inferior de los datos.
*   Q3: el tercer cuartil, que separa el 25% superior de los datos.
*   Luego, se calculará el IQR restando Q1 de Q3. Para definir los límites que nos ayudarán a identificar los "outliers" (valores atípicos), se usará la siguiente fórmula: `lower_bound = Q1 - 1.5 * IQR` and `upper_bound = Q3 + 1.5 * IQR`

In [ ]:
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("**************************")
print("Lower Bounds for Outliers:")
print("**************************")
print(lower_bound)
print("\n**************************")
print("Upper Bounds for Outliers:")
print("**************************")
print(upper_bound)

## **Distribuciones**

*   Revisar si los datos siguen una curva normal o si están muy sesgados hacia un lado.
*   Buscar patrones usando gráficos (Matplotlib, Seaborn o Plotly).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

plt.figure(figsize=(10, 6))
sns.histplot(df_numeric['DepDelayMinutes'], bins=50, kde=True, color='orange')
plt.title('Distribución de Minutos de Retraso en la Salida')
plt.xlabel('Minutos de Retraso en la Salida')
plt.ylabel('Frecuencia')
plt.xlim(-50, 250) # Adjusted x-limit for better visibility of main distribution and tail
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

plt.figure(figsize=(10, 6))
sns.histplot(df_numeric['ArrDelayMinutes'], bins=50, kde=True, color='purple')
plt.title('Distribución de Minutos de Retraso en la Llegada')
plt.xlabel('Minutos de Retraso en la Llegada')
plt.ylabel('Frecuencia')
plt.xlim(-50, 250) # Adjusted x-limit for better visibility of main distribution and tail
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

plt.figure(figsize=(10, 6))
sns.histplot(df_numeric['Distance'], bins=50, kde=True, color='teal')
plt.title('Distribución de la Distancia de Vuelo')
plt.xlabel('Distancia (millas)')
plt.ylabel('Frecuencia')
plt.xlim(0, 2000) # Adjusted x-limit to show most common distances without extreme long-haul flights
plt.show()

### Resumen del Análisis de Distribución

---
> Basado en los histogramas generados para `DepDelayMinutes`, `ArrDelayMinutes`, y `Distance`, las distribuciones:
> 
> *   **`DepDelayMinutes`** y **`ArrDelayMinutes`**: Ambas muestran distribuciones altamente sesgadas. Hay un pico muy fuerte en cero o cerca de cero, lo que indica que una gran mayoría de los vuelos salen y llegan a tiempo o incluso antes. Sin embargo, hay una larga cola que se extiende hacia la derecha, lo que muestra que, aunque son menos frecuentes, ocurren retrasos significativos. Estas distribuciones claramente no son normales y están fuertemente sesgadas positivamente.
> 
> *   **`Distance`**: La distribución de la distancia también está sesgada hacia la derecha, pero de manera menos dramática que las columnas de retraso. Muestra una mayor frecuencia para vuelos cortos, con la frecuencia disminuyendo gradualmente a medida que la distancia aumenta. Hay menos vuelos de larga distancia, lo que crea una cola hacia valores más altos. Esto es típico en conjuntos de datos de vuelos, donde muchas rutas de distancia corta a media son comunes, mientras que los vuelos internacionales muy largos son más raros.

## **Tipos de retrasos más comunes**

Vamos a crear un gráfico de barras para visualizar la frecuencia de cada causa de retraso. Esto nos mostrará qué tipos de retrasos son los más comunes en el conjunto de datos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

delay_cause_columns = [
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

delay_cause_columns_map = {
    0: 'Retraso de la Aerolínea',
    1: 'Retraso por Meteorológico',
    2: 'Retraso del Sistema Aéreo Nacional',
    3: 'Retraso de Seguridad',
    4: 'Retraso por Avión Tardío',
}


# Count the number of non-zero delays for each cause
frequency_of_causes = {}
for col in delay_cause_columns:
    # A flight is considered to have a delay cause if the delay is > 15
    frequency_of_causes[col] = df_numeric[df_numeric[col] > 15].shape[0]

# Convert to a pandas Series for easy plotting
frequency_series = pd.Series(frequency_of_causes).sort_values(ascending=False)

sns.set_style("whitegrid")

plt.figure(figsize=(12, 7))
sns.barplot(
    x=frequency_series.index,
    y=frequency_series.values,
    hue=frequency_series.index,
    palette='viridis',
    legend=False
)

plt.title('Frecuencia de Causas de Retraso (Retrasos No Nulos)')
plt.xlabel('Causa del Retraso')
plt.ylabel('Número de Vuelos Afectados')
#plt.xticks(rotation=45, ha='right')
plt.xticks(
    ticks=range(5),
    labels=[delay_cause_columns_map[i] for i in range(5)],
    rotation=45, ha='right'
)
plt.tight_layout()
plt.show()

---
> ### **Observaciones clave del gráfico**
>
> El gráfico de barras anterior muestra el número de vuelos afectados por cada causa específica de retraso, considerando solo los minutos de retraso no nulos. Esta visualización ayuda a identificar los tipos de retrasos más frecuentes en nuestro conjunto de datos. Podemos observar que `CarrierDelay`, `LateAircraftDelay` y `NASDelay` son las causas más frecuentes, afectando a miles de vuelos, mientras que `WeatherDelay` y especialmente `SecurityDelay` son menos frecuentes.

## **Probabilidad de retraso de vuelo por hora de salida programada**

In [ ]:
# Crear variable temporal 'hour' a partir de la hora programada
df_numeric["hour"] = df_numeric["CRSDepTime"] // 100  # solo la hora

# Variable objetivo 'delayed': 1 si el retraso en salida >= 15 min
df_numeric["delayed"] = (df_numeric["DepDelay"] >= 15).astype(int)

# Probabilidad de delay en la muestra
delay_rate = df_numeric["delayed"].mean()
print(f"Delay rate: {delay_rate:.4f}")

# Ver valores únicos de la variable 'delayed'
unique_values = df_numeric["delayed"].unique()
print("Valores únicos en 'delayed':", unique_values)

import matplotlib.pyplot as plt

# Agrupar por hora y calcular la probabilidad de retraso
hour_delay = (
    df_numeric.groupby("hour")["delayed"]
    .mean()
    .sort_index()
)

# Mostrar la tabla de probabilidades por hora
print(hour_delay)


plt.figure(figsize=(10,4))
hour_delay.plot(kind="line", marker="o")
plt.title("Probabilidad de Retraso de Vuelo por Hora de Salida Programada")
plt.xlabel("Hora del Día")
plt.ylabel("Probabilidad de Retraso")
plt.grid(True)
plt.show()

---
> ### **Observaciones clave del gráfico**
>
> *   **Baja Probabilidad de Retraso en la Temprana Mañana**: La probabilidad de retraso es generalmente baja durante las primeras horas de la mañana (por ejemplo, de 1 AM a 6 AM). Los vuelos programados para despegar en este intervalo tienden a tener menos posibilidades de ser retrasados.
>
> *   **Aumento de la Probabilidad a lo Largo del Día**: A medida que avanza el día, la probabilidad de que un vuelo sea retrasado aumenta de manera constante. Esta tendencia se vuelve más notable desde finales de la mañana hasta la tarde y el principio de la noche.
>
> *   **Pico de Probabilidad de Retraso en la Tarde/Noche**: Las mayores probabilidades de retraso se observan en la tarde y a inicios de la noche (aproximadamente entre las 4 PM y las 9 PM). Esto se debe a una combinación de factores: retrasos acumulativos de horas anteriores, mayor tráfico aéreo y posibles cambios climáticos.
>
> *   **Ligera Disminución a Última Hora de la Noche**: Hacia las últimas horas de la noche (por ejemplo, a partir de las 10 PM), la probabilidad de retraso tiende a disminuir nuevamente, probablemente debido a la reducción del tráfico aéreo.

## **Probabilidad de retraso según la distancia de vuelo**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Crear bins de distancia (5 quintiles)
df_numeric["distance_bin"] = pd.qcut(df_numeric["Distance"], q=5)

# Calcular probabilidad de retraso por rango de distancia
distance_delay = (
    df_numeric.groupby("distance_bin", observed=True)["delayed"]
    .mean()
)


print(distance_delay)

plt.figure(figsize=(8,4))
distance_delay.plot(kind="bar")
plt.title("Probabilidad de Retraso según la Distancia de Vuelo")
plt.ylabel("Probabilidad de Retraso")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()

---
> ### **Observaciones clave del gráfico**
>
> *   **Probabilidad Variada según las Distancias**: La probabilidad de retraso no es constante en todas las distancias de vuelo. Varía dependiendo de la longitud del vuelo.
>
> *   **Vuelos Cortos Tienen Menor Probabilidad de Retraso**: Los vuelos más cortos (rango de distancia (`15.999, 296.0`]) muestran la menor probabilidad de retraso (alrededor del 15%). Esto puede deberse a que las rutas más cortas son menos propensas a acumular retrasos o a enfrentar condiciones climáticas variadas.
>
> *   **Vuelos Intermedios y Largos**: Los vuelos en rangos de distancia intermedios y largos ((`296.0, 484.0`], (`484.0, 760.0`], (`760.0, 1099.0`], (`1099.0, 4983.0`]) tienden a tener probabilidades de retraso ligeramente más altas y más consistentes, que oscilan aproximadamente entre el 18% y el 19%.
>
> *   **Sin Relación Lineal Directa**: No hay una relación lineal clara y fuerte donde la probabilidad de retraso aumente o disminuya continuamente con la distancia. En cambio, parece fluctuar dentro de un cierto rango para vuelos de media a larga distancia después de una probabilidad inicial más baja para vuelos muy cortos.

## **Probabilidad de retraso por día de la semana**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Map day numbers to day names
days_map = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miercoles',
    3: 'Jueves',
    4: 'Viernes',
    5: 'Sabado',
    6: 'Domingo'
}


# Calcular probabilidad de retraso por día de la semana
dow_delay = (
    df_numeric.groupby("day_of_week")["delayed"]
    .mean()
    .sort_index()
)

# Graficar
plt.figure(figsize=(8,4))
dow_delay.plot(kind="bar")
plt.xticks(
    ticks=range(7),
    labels=[days_map[i] for i in range(7)],
    rotation=45
)
plt.title("Probabilidad de Retraso por Día de la Semana")
plt.xlabel("Día de la Semana")
plt.ylabel("Probabilidad de Retraso")
plt.grid(axis="y")
plt.show()

---
> **Observaciones clave del gráfico**
>*   **Mayor Probabilidad de Retraso los Lunes**: El gráfico probablemente muestra que los vuelos los lunes tienen una probabilidad relativamente más alta de ser retrasados. Esto podría deberse a una mayor demanda de viajes al comienzo de la semana laboral, a viajes de negocios o a retrasos acumulados del fin de semana.
>
>*   **Probabilidades Más Bajas en Mitad de Semana**: Típicamente, los días de mitad de semana como martes, miércoles y jueves pueden mostrar probabilidades de retraso ligeramente más bajas en comparación con el inicio o el final de la semana. Las operaciones pueden ser más fluidas con menos pasajeros y menos congestión.
>
>*   **Aumento de Probabilidades Hacia el Fin de Semana**: Los viernes pueden mostrar un aumento en la probabilidad de retraso nuevamente, a menudo atribuido al aumento de viajes por ocio y un mayor volumen de vuelos a medida que las personas viajan para el fin de semana.
>
>*   **Variabilidad en los Fines de Semana**: Sábados y domingos pueden tener patrones variados. A veces, los sábados muestran probabilidades más bajas debido a un tráfico de negocios más ligero, mientras que los domingos pueden aumentar debido a los viajes de regreso.



## **Top 10 Aerolíneas por Probabilidad de Retraso**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Crear una columna 'Marketing_Airline_Network' en df_numeric
df_numeric['Marketing_Airline_Network'] = df['Marketing_Airline_Network']

# Probabilidad de retraso por aerolínea (top 10)
alert_data = (
    df_numeric.groupby("Marketing_Airline_Network")["delayed"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

# Mostrar resultados
print(alert_data)

# Graficar
plt.figure(figsize=(8,4))
alert_data.plot(kind="bar")
plt.title("Top 10 Aerolíneas por Probabilidad de Retraso")
plt.ylabel("Probabilidad de Retraso")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()

## **Comprobación de variable objetivo binaria**

In [ ]:
df_numeric["delayed"].value_counts(normalize=True)

hour_delay = (
    df_numeric.groupby("hour")["delayed"]
    .mean()
)

hour_std = hour_delay.std()

print(f"La probabilidad de retraso de los vuelos en promedio\na lo largo de las diferentes horas del día: {hour_std:.2%}")

---
>
> Este criterio mide cuánto cambia la probabilidad de retraso a lo largo del día;
> si la variación es alta, significa que la hora del vuelo influye de forma real
> en los retrasos, y el ratio entre datasets nos permite comparar cuál captura
> mejor ese patrón temporal sin necesidad de entrenar ni optimizar modelos.
>
> Al medir la variación de la probabilidad de retraso según la hora, verificamos
> que la hora del vuelo tiene impacto real, y usando el ratio podemos decidir
> qué dataset conserva mejor esa señal predictiva sin incurrir en mayor coste
> computacional.



**Advertencia**

Las variables:

* CarrierDelay
* WeatherDelay
* NASDelay
* SecurityDelay
* LateAircraftDelay

ocurren después del retraso, por lo que:
# Crear variable temporaimport matplotlib.pyplot as plt
import pandas as pd

# Crear una columna 'Marketing_Airline_Network' en df_numeric
df_numeric['Marketing_Airline_Network'] = df['Marketing_Airline_Network']

# Probabilidad de retraso por aerolínea (top 10)
alert_data = (
    df_numeric.groupby("Marketing_Airline_Network")["delayed"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

# Mostrar resultados
print(alert_data)

# Graficar
plt.figure(figsize=(8,4))
alert_data.plot(kind="bar")
plt.title("Top 10 Aerolíneas por Probabilidad de Retraso")
plt.ylabel("Probabilidad de Retraso")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()l 'hour' a partir de la hora programada
df_numeric["hour"] = df_numeric["CRSDepTime"] // 100  # solo la hora

# Variable objetivo 'delayed': 1 si el retraso en salida >= 15 min
df_numeric["delayed"] = (df_numeric["DepDelay"] >= 15).astype(int)

# Probabilidad de delay en la muestra
delay_rate = df_numeric["delayed"].mean()
print(f"Delay rate: {delay_rate:.4f}")

# Ver valores únicos## **Top 10 Aerolíneas por Probabilidad de Retraso** de la variable 'delayed'
unique_values = df_numeric["delayed"].unique()
print("Valores únicos en 'delayed':", unique_values)

import matplotlib.pyplot as plt

# Agrupar por hora y calcular la probabilidad de retraso
hour_delay = (
    df_numeric.groupby("hour")["delayed"]
    .mean()
    .sort_index()
)

# Mostrar la tabla de probabilidades por hora
print(hour_delay)


plt.figure(figsize=(10,4))
hour_delay.plot(kind="line", marker="o")
plt.title("Probabilidad de Retraso de Vuelo por Hora de Salida Programada")
plt.xlabel("Hora del Día")
plt.ylabel("Probabilidad de Retraso")
plt.grid(True)
plt.show()
Son válidas para análisis descriptivo

NO son válidas para un modelo predictivo ex-ante

Para predicción realista, excluirlas y entrenar dos modelos:

* Modelo operacional (ex-ante)
* Modelo explicativo (post-evento)

Si se desea, se puede reestructurar el pipeline en dos variantes formales (predictivo vs explicativo), lo que es muy valorado en contextos académicos.


## Variables permitidas (ex-ante)

**Temporales / calendario**
* Year
* Month
* DayofMonth
* day_of_week (derivada)

**Programación**
* CRSDepTime
* CRSElapsedTime
* Distance
* DistanceGroup

**Históricas / estructurales**
* Marketing_Airline_Network
* OriginCityName
* DestCityName

**❌ Variables excluidas (post-evento)**

* DepDelayMinutes
* ArrDelayMinutes
* TaxiOut, TaxiIn
* TODAS las *Delay (Carrier, Weather, etc.)

## Dataset predictivo

In [ ]:
import numpy as np
'''
    "ArrDelayMinutes",
    "OriginCityName",
    "DestCityName",
    "Marketing_Airline_Network",
    "DistanceGroup",
    "Distance",
    "distancia_km",
    "DayofMonth",
    "Year",
    "Month",
    "day_of_week",
    "day_of_year",
    "day_name",
    "is_weekend",
    "week_of_year",
    "quarter",
    "hour",
    "CRSDepTime",
    "CRSDepTime_minutes",
    "CRSDepTime_sin",
    "CRSDepTime_cos"
    "CRSElapsedTime",
    "DepDelayMinutes",
    "TaxiOut",
    "TaxiIn",
    "AirTime",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay",
    "delayed"
'''
df_pred = df_model[[
    "CRSDepTime_sin",
    "CRSDepTime_cos"
    "CRSElapsedTime",
    "distancia_km",
    "DistanceGroup",
    "Marketing_Airline_Network",
    "OriginCityName",
    "DestCityName",
    "DayofMonth",
    "Year",
    "Month",
    "hour",
    "day_of_week",
    "is_weekend",
    "quarter",
    "delayed"
]].copy()



## Codificación categórica

In [ ]:
import pandas as pd

# Calcular la media global de 'delayed' en el conjunto de entrenamiento para imputar categorías no vistas
global_mean_delayed_train = df_pred['delayed'].mean()

# Calcular target encodings utilizando solo el conjunto de entrenamiento
origen_map = df_pred.groupby('origen')['delayed'].mean()
destino_map = df_pred.groupby('destino')['delayed'].mean()
aerolinea_map = df_pred.groupby('aerolinea')['delayed'].mean()

# 2. Aplicar estos mappings a las columnas de X_train y X_test
#    Se usa .map() y .fillna() para manejar categorías no vistas, imputándolas con la media global de y_train.
df_pred['origen'] = df_pred['origen'].map(origen_map).fillna(global_mean_delayed_train)
df_pred['destino'] = df_pred['destino'].map(destino_map).fillna(global_mean_delayed_train)
df_pred['aerolinea'] = df_pred['aerolinea'].map(aerolinea_map).fillna(global_mean_delayed_train)

In [ ]:
# Codificación categórica para alta cardinalidad
# Se reemplaza OneHotEncoder por Target Encoding con regularización

from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer

cat_cols = [
    "day_of_week"
]

num_cols = [c for c in df_pred.columns if c not in cat_cols + ["delayed"]]

preprocess = ColumnTransformer(
    transformers=[
        (
            "cat",
            TargetEncoder(
                cols=cat_cols,
                smoothing=10
            ),
            cat_cols
        ),
        ("num", "passthrough", num_cols)
    ]
)


## **Modelo recomendado**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe_pred = Pipeline(
    steps=[
        ("prep", preprocess),
        ("model", LogisticRegression(
            max_iter=1000,
            class_weight="balanced"
        ))
    ]
)

## **Columnas a eliminar por redundancia o fuga de datos**

Las siguientes columnas se recomiendan eliminar del `df_numeric` debido a que su información ya está contenida en otras características más refinadas o directamente duplicadas:

*   **`Year`, `Month`, `DayofMonth`**: La información de la fecha ya está capturada de forma más útil y granular en las características `day_of_year`, `week_of_year`, `quarter`, `day_of_week` e `is_weekend`.
*   **`CRSDepTime`**: La hora de salida programada está mejor representada por las características cíclicas `CRSDepTime_minutes`, `CRSDepTime_sin` y `CRSDepTime_cos`, que manejan la naturaleza cíclica del tiempo de forma más adecuada para los modelos.
*   **`Distance`**: Esta columna se ha convertido y ahora está representada por `distancia_km`, por lo que la `Distance` original (en millas) es redundante.
*   **`distance_bin`**: Esta es una versión binned (categorizada) de la distancia. Si ya estamos utilizando `distancia_km` (que es una representación continua) y/o `DistanceGroup` (otra categorización), `distance_bin` puede ser redundante.
*   **`Marketing_Airline_Network`**: Después de aplicar la codificación por objetivo a esta columna para crear la característica `aerolinea` (flotante), la columna original `Marketing_Airline_Network` (tipo `object`) se vuelve redundante como característica.
*   **`hour`**: Esta columna fue derivada de `CRSDepTime`. Si se utilizan las características cíclicas `CRSDepTime_sin` y `CRSDepTime_cos`, la columna `hour` se vuelve redundante o menos informativa, ya que las características cíclicas capturan mejor la relación temporal.
*   **`DistanceGroup`**: Aunque es una categorización de la distancia, si ya tenemos `distancia_km` que ofrece una granularidad continua y es la preferida, `DistanceGroup` puede considerarse redundante para evitar el uso de múltiples representaciones de la misma información de distancia. Se recomienda conservar `distancia_km`.
*   **`DepDelayMinutes`**, **`CRSDepTime`**, **`DepDelay`**, **`DepTime`**, **`TaxiIn`**, **`ArrDelay`**, **`ArrTime`**, **`ArrDelayMinutes`**, **`ActualElapsedTime`**, **`AirTime`**, **`CarrierDelay`**, **`WeatherDelay`**, **`NASDelay`**, **`SecurityDelay`**, **`LateAircraftDelay`**, **`TaxiOut`**, **`WheelsOff`**, **`WheelsOn`**: Las siguientes columnas se recomiendan eliminar debido a que su información no se conoce antes de que el vuelo ocurra y es un resultado directo del mismo.

### **Separar la Variable Objetivo**

Ahora que se han agregado los nombres de las ciudades, el siguiente paso es identificar explícitamente todas las columnas categóricas y numéricas en el DataFrame `df_numeric`, para preparar los pasos de procesamiento subsecuentes como la codificación y la escalación.

`delayed` es la variable objetivo (si un vuelo se retrasa o no). No debe ser utilizada como una característica de entrada para el modelo, sino como la variable que el modelo intentará predecir. Por lo tanto, debe ser separada del conjunto de características antes del entrenamiento.


### **Dividir los datos en conjuntos de entrenamiento, validacón y prueba**

Para comprender si el modelo realmente está aprendiendo de los datos, aplicamos el estrategia `holdout`, separando los datos en tres partes: datos de entrenamiento, validación y prueba:

*   El **conjunto de entrenamiento** se utiliza para entrenar los modelos. A partir de este conjunto, los modelos identifican patrones en los datos.
*   El **conjunto de validación** se emplea para evaluar el desempeño de diferentes modelos con datos nuevos que no fueron utilizados en el entrenamiento.
*   El **conjunto de prueba** se mantiene separado desde el inicio para simular datos del mundo real. No se utiliza en ninguna etapa del entrenamiento ni de la validación, sirviendo como una estimación de la capacidad del modelo elegido para generalizar y predecir nuevos datos.

Realizaremos la división por año de los datos entre entrenamiento, validación y prueba dado a que:
* Simula el escenario real: entrenar con pasado → predecir futuro
* Evita fuga de información temporal.

In [ ]:
train = df_pred[df_pred["Year"] <= 2018]
test  = df_pred[df_pred["Year"] > 2018]

valid = train[train["Year"] == 2018]
train = train[train["Year"] < 2018]

X_train = train.drop(columns="delayed")
y_train = train["delayed"]

X_test = test.drop(columns="delayed")
y_test = test["delayed"]

pipe_pred.fit(X_train, y_train)

## Evaluación

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Predicción usando el pipeline completo (preprocesamiento + modelo)
y_pred = pipe_pred.predict(X_test)
y_proba = pipe_pred.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

## 9. Interpretabilidad

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Extraer coeficientes del modelo dentro del pipeline
coef = pipe_pred.named_steps["model"].coef_[0]

# Recuperar nombres de variables (numéricas + categóricas codificadas)
feature_names = (
    num_cols +
    cat_cols
)

importance = (
    pd.Series(coef, index=feature_names)
      .sort_values()
)

plt.figure(figsize=(8, 6))
importance.plot(kind="barh")
plt.title("Importancia de variables (modelo predictivo ex-ante)")
plt.xlabel("Coeficiente")
plt.tight_layout()
plt.show()


## 10. Visualización de negocio

Probabilidad de retraso por día de la semana

In [ ]:
import matplotlib.pyplot as plt

# Probabilidad de retraso por día de la semana (modelo predictivo)
dow_delay = (
    df_pred
        .groupby("day_of_week")["delayed"]
        .mean()
        .reindex(range(7))  # asegura orden lunes → domingo
)

plt.figure(figsize=(8, 4))
dow_delay.plot(kind="bar")
plt.title("Probabilidad de Retraso por Día de la Semana")
plt.xlabel("Día de la Semana")
plt.ylabel("Probabilidad de Retraso")
plt.grid(axis="y")
plt.tight_layout()
plt.show()


## 11. Reproducibilidad

**requirements.txt**

pandas>=1.5
numpy>=1.23
scikit-learn>=1.3
matplotlib>=3.7
seaborn>=0.13

## GitHub

* Código
* Notebooks
* Sin datasets grandes

## 12. Extensiones naturales

* XGBoost / LightGBM
* Validación temporal (por año)
* SHAP
* Segmentación por aerolínea
* Cost-sensitive learning

**Cierre**

Este pipeline es:

* Escalable
* Reproducible
* Compatible con Kaggle y Colab
* Apto para análisis académico y productivo
* Puede extenderse fácilmente a modelos más complejos o a análisis causal.